# Visualization of changes by specle variability using Sentinel-1 data

### **R**apid and **EA**sy **C**hange detection in radar **TI**me-series by **V**ariation coefficient

![REACTIV logo](https://raw.githubusercontent.com/elisekoeniguer/REACTIV/master/REACTIV.png)  

##### display a stack of SAR images with change detection highlighting

_Colored visualization of multitemporal data for change detection: issues and methods_, Elise Colin Koeniguer et al., EUSAR 2018  
[github link](https://github.com/elisekoeniguer/REACTIV)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import datetime

from scipy.special import gamma
from skimage import color

from eolearn.io import S1IWWCSInput
from sentinelhub import SHConfig
from eolearn.core import EOTask, EOPatch, FeatureType

from sentinelhub import BBox, CRS

In [ ]:
def specle_variability(eopatch,
                       data_feature=(FeatureType.DATA, 'IW_VV'),
                       mask_feature=(FeatureType.MASK, 'IS_DATA')):
    
    eopatch_size = eopatch[data_feature][0][...,0].shape
    time_delta = np.max(eopatch.timestamp) - np.min(eopatch.timestamp)
    min_time = np.min(eopatch.timestamp)
    time = (np.array(eopatch.timestamp)-min_time)/time_delta

    masked_eopatch = np.ma.array(eopatch[s.data_feature].squeeze(), 
                                 mask=np.logical_not(eopatch[mask_feature].squeeze()))

    mb1 = masked_eopatch.mean(axis=0) #M1
    mb2 = np.ma.power(masked_eopatch, 2).mean(axis=0) #M2
    kmax = time[masked_eopatch.argmax(axis=0).ravel()].reshape(eopatch_size).astype(np.float32)
    imax = eopatch[data_feature].squeeze().max(axis=0) #Imax

    R = np.sqrt(mb2-mb1**2)/mb1

    gam = R.mean()
    a = 0.991936+0.067646*gam-0.098888*gam**2 -0.048320*gam**3
    b = 0.001224-0.034323*gam+4.305577*gam**2-1.163498*gam**3
    L = a/b;

    CV = np.sqrt((L*gamma(L)**2/(gamma(L+0.5)**2))-1); # theretical mean value
    num = (L*gamma(L)**4.*(4*(L**2)*gamma(L)**2-4*L*gamma(L+1/2)**2-gamma(L+1/2)**2));
    den = (gamma(L+1/2)**4.*(L*gamma(L)**2-gamma(L+1/2)**2));
    alpha = 1/4*num/den; # theretical standard deviation value

    R = (R-CV)/(alpha/np.sqrt(np.count_nonzero(eopatch[data_feature], axis=0).squeeze()))/10.0+0.25; 
    R = np.clip(R.data, a_min=0, a_max=1);   # Cast Coefficient of Varation R max to 1.

    threshold = np.mean(masked_eopatch[0]) + 15*np.std(masked_eopatch[0])
    I = np.clip(imax/threshold, a_max=1, a_min=0); # normalize Intensity to threshold. 

    hsv = np.stack([kmax, R.data, I], axis=2).astype(np.float32)

    return hsv

## Practice

The above method seems suitable to be put into an `EOTask`. Let's make one!

<div class="alert alert-success">
 <b>EXERCISE</b>:


* Create an [EOTask](eolearn_basics.ipynb#EOTask) from the `specle_variability` method. Think if the resulting data is temporal or timeless (in terms of `FeatureType`). 
* Run the task on `data/sentinel1_sample` and produce an image. Hint: convert Hue/Saturation/Value to RGB image using `color.hsv2rgb(hsv)` from `skimage`
* Try to combine lessons learned in **Sentinel-Hub and other datasources** section, download Sentinel-1 data for year 2018 over these coordinates (BBOX here), use '40m' for `resolution` parameter and then run the task to produce an image. Do you recognize what is going on (and where)?
 
</div>

<hr>
EOTask wrapping the `specle_variability` method

In [ ]:
# %load ../solutions/sentinel1_task.py

<hr>
Produce an image using above EOTask

In [ ]:
# %load ../solutions/sentinel1_image.py

<hr>
Download S1 data, run task, produce image

In [ ]:
# %load ../solutions/sentinel1_pipeline.py